In [1]:
from IPython.display import clear_output # clear output
from PIL import Image # image processing
import numpy as np # linear algebra
from tqdm import tqdm # progress bar
import os # file handling
from pymongo import MongoClient

In [2]:
# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/Scibio")  # Remplacez par l'URL de votre serveur MongoDB
db = client["Scribio"]  # Nom de la base de données
collection = db["Dataset"]  # Nom de la collection

In [3]:
# Transformation des photos en dataframe
def photo_to_dataframe(chemins_photos: str):
    """Transforme les photos en tableau de pixels et les sauvegarde dans un fichier csv

    Args:
        chemins_photos (str): Chemin des photos (on veut le dossier contenant les photos)
        label (str): label de la photo (ce qui va être prédit)
        separation (str): séparation des données (train, test, validation)
    """
    tab_img = [] # Initialisation de la liste des images (dictionnaire avec les infos des images)
    
    # Split du chemin pour recupérer les informations
    chemin_split = chemins_photos[0].split("\\")
    
    # Affichage des dossiers en cours de traitement 
    print("..\\" + chemin_split[1]) # Dataset
    print("   \\" + chemin_split[2]) # dataset, datasetDigit, datasetSmall
    print("    \\" + chemin_split[3]) # Répartition : test, train, validation
    print("     \\" + chemin_split[4]) # label (26 + 9 + 26)
    
    # Parcours et transformation des photos avec barre de progression 
    with tqdm(total=len(chemins_photos), desc="Proccessing" , position=0, leave=True) as pbar:
        for chemin in chemins_photos: # Parcours des chemins des photos
            
            pbar.set_postfix(img=chemin[-10:]) # Mettre à jour la barre de progression avec des informations supplémentaires
            
            img = Image.open(chemin) # 300 x 300
            nouvelle_taille = (50, 50) # 2500 
            img_redimensionnee = img.resize(nouvelle_taille) # Redimensionner l'image
            nb_image = img_redimensionnee.convert('L') # Convertion en noir et blanc
            
            #  Récupération de la liste des pixels
            tab = []
            for i in range(nb_image.size[1]):
                row = []
                for y in range(nb_image.size[0]):
                    # Récupérer la couleur du pixel
                    row.append(nb_image.getpixel((y, i)))
                tab.append(row)
                
            tab_numpy = np.array(tab) / 255 # mise à l'échelle des données
            tab_numpy_flatten = tab_numpy.flatten() # Transformation en tableau 1D
            tab_img.append({  # Ajout du tableau dans la liste
                "Label": chemin_split[4],
                "Repartition": chemin_split[3],
                "Chemin": chemin.replace('\\', '/'),
                "Tableau_Pixels": tab_numpy_flatten.tolist()
            })
    
            pbar.update(1) # Mettre à jour la barre de progression
    
    print("Envoie dans la base de données ...") # Affichage de la progression
    collection.insert_many(tab_img) # Insertion des données dans la base de données
    
    pbar.close() # Fermer la barre de progression
    
    clear_output()  # Effacer l'output précédent (sinon pb d'affichage de la barre de progression)

In [4]:
# Parcours des dossiers et sous-dossiers pour recréer le dataset (recursion)
def recreate_dataset(chemin: str):
    """Réecriture du dataset: transformation des photos en tableau de pixels et les sauvegarde dans un fichier csv
    Cette fonction est récursive et doit être executer sur un dossier contenant la même arborescence que le dossier ../Dataset

    Args:
        chemin (str): chemin du dataset
    """
    dossiers = [f for f in os.listdir(chemin) if os.path.isdir(os.path.join(chemin, f))]
    if len(dossiers) == 0: # Si aucun dossier n'est trouvé, on est dans le cas de base
        fichiers_png = [os.path.join(chemin, f) for f in os.listdir(chemin) if f.endswith('.png')] # on recupère et stocke les fichiers .png
        photo_to_dataframe(fichiers_png) # On transforme les photos en tableau en df qu'on save dans une bdd
    else : # Sinon, on continue de parcourir les dossiers
        for dossier in dossiers:
            recreate_dataset(os.path.join(chemin, dossier))

In [ ]:
# Appel de la fonction de parcours des dossiers
recreate_dataset("..\Dataset")
# Temps : 145m 20.9s => 2h 25m 20.9s